# Artifacts analysis

In [1]:
from deepfix.core.agents.coordinators import ArtifactAnalysisCoordinator
from deepfix.core.config import MLflowConfig,ArtifactConfig

d:\workspace\repos\deepfix\.venv\Lib\site-packages\deepchecks\core\serialization\dataframe\html.py:16: UserWarning:

pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.



### Ingest Image Classification Dataset

In [3]:
from deepfix.core.pipelines.factory import DatasetLoggingPipeline
from deepfix.zoo.datasets.foodwaste import load_train_and_val_datasets

In [ ]:
# Load image datasets
train_data, val_data = load_train_and_val_datasets(
    image_size=448,
    batch_size=8,
    num_workers=4,
    pin_memory=False,)

In [ ]:
dataset_name="cafetaria-foodwaste-lstroetmann"

dataset_logging_pipeline = DatasetLoggingPipeline(dataset_name=dataset_name,
                                                train_test_validation=True,
                                                data_integrity=True,
                                                batch_size=8,
                                                overwrite=False # True -> i.e. delete and re-create
                                                )

dataset_logging_pipeline.run(train_data=train_data,
                            test_data=val_data,
                        )

### Analyze Dataset

In [3]:
mlflow_config = MLflowConfig()
artifact_config = ArtifactConfig(dataset_name="cafetaria-foodwaste-lstroetmann",
                                load_dataset_metadata=True,
                                load_checks=False,
                                load_model_checkpoint=False,
                                load_training=False,
                                download_if_missing=True,
                                cache_enabled=True
                            )

In [4]:
coordinator = ArtifactAnalysisCoordinator.from_config(mlflow_config=mlflow_config, 
                                            artifact_config=artifact_config,
                                            env_file="D:/workspace/repos/deepfix/.env")
result = coordinator.run()

In [5]:
print(result.to_text())


DEEPFIX ANALYSIS RESULT
Dataset: cafetaria-foodwaste-lstroetmann
Run ID: None
Summary of the analysis:
This dataset exhibits multiple critical flaws that fundamentally undermine its suitability for computer vision modeling. The most severe issue is the completely non-representative train-test split, evidenced by both statistical analysis and deepchecks validation. 

**Core Problems:**
1. **Catastrophic Label Distribution Mismatch**: 75% of test set labels are absent from the training set, with specific labels ('4', '1', '37') appearing exclusively in testing. This makes performance metrics completely unreliable as the model is being tested on classes it never encountered during training.

2. **Severe Statistical Drift**: Multiple image properties show significant differences between sets - brightness (0.43 KS), contrast (0.5 KS), and color channel intensities (0.82-0.96 KS). This indicates the train and test data likely come from different sources or collection conditions.

3. **Extre

In [5]:
#print(result.summary)

In [6]:
#ctx = result.context

In [7]:
#print(ctx.to_summary())